In [ ]:
!pip install cv

In [ ]:
##  capture Image from camera and create dataset

In [9]:
import cv2
import os

def generate_dataset():
    face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

    def face_cropped(img):
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray, 1.3, 5)
        if len(faces) == 0:
            return None
        for (x, y, w, h) in faces:
            cropped_face = img[y:y+h, x:x+w]
            return cropped_face

    cap = cv2.VideoCapture(0)
    id = 1  # here you can change id for uniqe user
    img_id = 0

    while True:
        ret, frame = cap.read()
        if face_cropped(frame) is not None:
            img_id += 1
            face = cv2.resize(face_cropped(frame), (200, 200))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            data_path = "data/user." + str(id) + "." + str(img_id) + ".jpg"
            cv2.imwrite(data_path, face)
            cv2.putText(face, str(img_id), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
            cv2.imshow("Cropped face", face)

        if cv2.waitKey(1) == 13 or img_id == 200:  # 13 is Enter key
            break

    cap.release()
    cv2.destroyAllWindows()
    print("Collecting sample pictures completed.")

generate_dataset()


In [24]:
!pip install pillow

In [ ]:
!pip uninstall opencv-python

In [2]:
!pip install opencv-contrib-python

  Using cached opencv_contrib_python-4.11.0.86-cp37-abi3-macosx_13_0_arm64.whl.metadata (20 kB)
Using cached opencv_contrib_python-4.11.0.86-cp37-abi3-macosx_13_0_arm64.whl (46.3 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 25.7 MB/s eta 0:00:00a 0:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.2.6 which is incompatible.
contourpy 1.2.0 requires numpy<2.0,>=1.20, but you have numpy 2.2.6 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.6 which is incompatible.


In [12]:
import os
import cv2
import numpy as np
from PIL import Image

def train_classifier(data_dir):
    path = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    faces = []
    ids = []

    for image in path:
        img = Image.open(image).convert('L')  # Convert to grayscale
        imageNp = np.array(img, 'uint8')
        id = int(os.path.split(image)[1].split(".")[1])  # Extract ID from filename
        faces.append(imageNp)
        ids.append(id)

    ids = np.array(ids)

    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.train(faces, ids)
    clf.write("classifier.xml")

# Call the function
train_classifier("data")


In [1]:
import cv2

def draw_boundary(img, classifier, scaleFactor, minNeighbors, color, text, clf):
    gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    features = classifier.detectMultiScale(gray_image, scaleFactor, minNeighbors)
    coords = []

    for (x, y, w, h) in features:
        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
        id, pred = clf.predict(gray_image[y:y + h, x:x + w])
        confidence = int(100 * (1 - pred / 300))

        if confidence > 77:
            if id == 1:
                name = "Amarjeet"
            elif id == 2:
                name = "dhruv"
            elif id == 3:
                name = "Krishna"
            else:
                name = "Unknown"
        else:
            name = "Unknown"

        cv2.putText(img, name, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
        coords = [x, y, w, h]
    return coords

def recognize(img, clf, faceCascade):
    draw_boundary(img, faceCascade, 1.1, 10, (0, 0, 0), "Face", clf)
    return img

# Load classifier and recognizer
faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
clf = cv2.face.LBPHFaceRecognizer_create()
clf.read("classifier.xml")
url ="http://192.168.1.109:8080/stream"
# Start video capture
video_capture = cv2.VideoCapture(url)
while True:
    ret, img = video_capture.read()
    if not ret:
        break
    img = recognize(img, clf, faceCascade)
    cv2.imshow("Face Detection", img)

    if cv2.waitKey(1) == 13:  # 13 is Enter key
        break

video_capture.release()
cv2.destroyAllWindows()


2025-05-21 17:57:13.365 python[45175:554431] +[IMKClient subclass]: chose IMKClient_Modern
2025-05-21 17:57:13.365 python[45175:554431] +[IMKInputSession subclass]: chose IMKInputSession_Modern
